In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

2024-01-29 00:42:26.036390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/hugo/tensorflow-metal-test/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
#Preprocessing --- Goal to make the data compatible for CNN input
# In tensorflow, we can achieve the same using ImageGenerators

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)

In [3]:
#Pass the images through the generator
trainImageData = train_generator.flow_from_directory("../data/training",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 


testImageData = train_generator.flow_from_directory("../data/testing",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [4]:
#Given dataset is a balanced dataset

trainImageData.class_indices

{'flip': 0, 'notflip': 1}

In [6]:
trainImageData.image_shape

(64, 64, 3)

# Loading ResNet from keras to perform transfer learning

In [36]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [37]:
# Importing the ResNet model but discarding the last 1000 neuron layer for adjustment to our dataset
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape=(64, 64, 3))

In [38]:
# Freeze the base_model so we keep the pre-trained structure
base_model.trainable = False

In [39]:
# Adjusting the model to our dataset

model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [40]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 2, 2, 2048)        58370944  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 1024)              8389632   
                                                                 
 dense_7 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 66,761,601
Trainable params: 8,390,657
Non-trainable params: 58,370,944
_________________________________________________________________


In [41]:
#Compile

model.compile(optimizer="adam",
              loss="binary_crossentropy", #For multi-class classification: categorical_crossentropy | sparse_categorical_crossentropy
              metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

In [42]:
len(trainImageData.filenames) // trainImageData.batch_size

74

In [43]:
# Fit the model
model.fit(trainImageData,
          validation_data=testImageData,
          epochs=100,
          steps_per_epoch = len(trainImageData.filenames) // trainImageData.batch_size,
          validation_steps= len(testImageData.filenames) // testImageData.batch_size)

Epoch 1/100


2024-01-29 01:27:46.990184: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 01:27:49.573705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-29 01:27:49.827812: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


74/74 [==============================] - ETA: 0s - loss: 0.9685 - f1_score: 0.6089

2024-01-29 01:28:24.787296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 01:28:25.422821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 49s 582ms/step - loss: 0.9685 - f1_score: 0.6089 - val_loss: 0.6365 - val_f1_score: 0.6204
Epoch 2/100
74/74 [==============================] - 38s 511ms/step - loss: 0.6355 - f1_score: 0.6637 - val_loss: 0.6198 - val_f1_score: 0.7223
Epoch 3/100
74/74 [==============================] - 38s 510ms/step - loss: 0.6243 - f1_score: 0.6786 - val_loss: 0.6482 - val_f1_score: 0.4059
Epoch 4/100
74/74 [==============================] - 39s 523ms/step - loss: 0.6270 - f1_score: 0.6604 - val_loss: 0.6085 - val_f1_score: 0.6450
Epoch 5/100
74/74 [==============================] - 38s 520ms/step - loss: 0.6115 - f1_score: 0.6920 - val_loss: 0.7190 - val_f1_score: 0.2081
Epoch 6/100
74/74 [==============================] - 38s 520ms/step - loss: 0.6000 - f1_score: 0.7078 - val_loss: 0.5900 - val_f1_score: 0.6990
Epoch 7/100
74/74 [==============================] - 38s 513ms/step - loss: 0.5813 - f1_score: 0.7104 - val_loss: 0.6090 - val_f1_score: 0.7537
Epoc

74/74 [==============================] - 38s 517ms/step - loss: 0.3798 - f1_score: 0.8395 - val_loss: 0.4481 - val_f1_score: 0.8438
Epoch 58/100
74/74 [==============================] - 38s 514ms/step - loss: 0.3881 - f1_score: 0.8426 - val_loss: 0.4266 - val_f1_score: 0.8470
Epoch 59/100
74/74 [==============================] - 38s 511ms/step - loss: 0.3886 - f1_score: 0.8327 - val_loss: 0.4445 - val_f1_score: 0.7993
Epoch 60/100
74/74 [==============================] - 38s 513ms/step - loss: 0.4071 - f1_score: 0.8306 - val_loss: 0.5318 - val_f1_score: 0.7558
Epoch 61/100
74/74 [==============================] - 38s 515ms/step - loss: 0.3602 - f1_score: 0.8477 - val_loss: 0.4444 - val_f1_score: 0.8429
Epoch 62/100
74/74 [==============================] - 38s 511ms/step - loss: 0.3615 - f1_score: 0.8484 - val_loss: 0.4330 - val_f1_score: 0.8470
Epoch 63/100
74/74 [==============================] - 38s 510ms/step - loss: 0.3482 - f1_score: 0.8636 - val_loss: 0.4088 - val_f1_score: 0.841

In [44]:
model.save('../models/ResNet.keras')
#model.save('monReader.keras_tf', save_format='tf')

### The ResNet re-trained model to our dataset is 335MB, being the largest model and so a model that will NOT fit in a smartphone app! 